In [167]:
import numpy as np
import pandas as pd
import tensorflow as tf
from geopy import distance
import geopandas
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

import keras 
from keras.models import Sequential
from keras.layers import Dense, Dropout
import tensorflow as tf
from keras.utils import np_utils
from keras.models import load_model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout


from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.feature_selection import RFECV, RFE

import math
from sklearn.preprocessing import FunctionTransformer
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler

from sklearn.metrics import mean_squared_error, classification_report, confusion_matrix, accuracy_score, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV

plt.rcParams['figure.figsize'] = (15, 9)
plt.style.use('Solarize_Light2')

>### Load Model file.

In [168]:
#model_file = './models/my_model.h5'
#model = load_model(model_file)
model=keras.models.load_model('azimsBRAIN.h5')

>## Prediction Model for external data

In [169]:
""" The Function below is asuming the external test data is a csv file, with one or more rows."""

""" The function is assuming that the test data file has the same structure and data types as the data that the model was trained on, 
    so it can perform the same preprocessing steps before making predictions. """

def load_process_predict3(data_file):
    
    # Load the new data file as a Pandas DataFrame
    data = pd.read_csv(data_file)
    print('Test Data Shape (before processing): ', data.shape)
    
    # Separate out the target variable if it exists
    if 'is_fraud' in data.columns:
        target = data['is_fraud']
        data = data.drop('is_fraud', axis=1)
    else:
        target = None
    
    # Preprocess the new data using the same steps as the training data
    data['trans_date_trans_time'] = pd.to_datetime(data['trans_date_trans_time'])
    data['year'] = data['trans_date_trans_time'].dt.year
    data['month'] = data['trans_date_trans_time'].dt.month
    data['day'] = data['trans_date_trans_time'].dt.day
    data['hour'] = data['trans_date_trans_time'].dt.hour
    data['minute'] = data['trans_date_trans_time'].dt.minute
    data['second'] = data['trans_date_trans_time'].dt.second
    
    data.drop('trans_date_trans_time', axis=1, inplace=True)
    
    data['Names'] = data['first'] + '-' + data['last']
    data.drop(['first', 'last'], axis=1, inplace=True)
    
    data['dob'] = pd.to_datetime(data['dob'])
    data['age'] = (pd.Timestamp.now(tz=None) - data['dob']).astype('<m8[Y]')
    data.drop('dob', axis=1, inplace=True)

    #convert 'amt' column to log
    #data['amt']=np.log(data['amt'])

    data.drop(columns={'zip', 'merch_lat', 'merch_long', 'street', 'job', 'trans_num', 'Names'}, inplace=True)
    
    if target is not None:
        # Apply label encoding to categorical columns
        le = LabelEncoder()
        for col in data.select_dtypes(exclude=[np.number]).columns:
            data[col] = le.fit_transform(data[col])
        
        # Scale the input features
        sc = StandardScaler()
        data = sc.fit_transform(data)
        
        # Make predictions using the loaded model
        predictions = (model.predict(data) > 0.5).astype("int32")

        # Print classification report and confusion matrix
        print(classification_report(target, predictions))
        print(confusion_matrix(target, predictions))
        #print('Accuracy', accuracy_score(target, predictions))
    else:
        # Apply label encoding to categorical columns
        le = LabelEncoder()
        for col in data.select_dtypes(exclude=[np.number]).columns:
            data[col] = le.fit_transform(data[col])
        
        # Scale the input features
        sc = StandardScaler()
        data = sc.fit_transform(data)
        
        # Make predictions using the loaded model
        predictions = (model.predict(data) > 0.5).astype("int32")
        
        return print('Predicted Label: ', predictions)

>## Create Test file

In [170]:
# Load the original data
original = pd.read_csv('Assignemnt4data.csv')
#original=original[original['amt']>100]

>## Test the function

### Test File with Target Column

In [171]:
test_data_fraud = original[original['is_fraud'] == 1].sample(n=5, random_state=None)
test_data_nonfraud = original[original['is_fraud'] == 0].sample(n=5, random_state=None)
test_data = pd.concat([test_data_fraud, test_data_nonfraud], axis=0)
test_data.to_csv('With_Target.csv', index=False)

In [172]:
load_process_predict3('With_Target.csv')

Test Data Shape (before processing):  (10, 22)
1/1 [==============================] - 0s 52ms/step
              precision    recall  f1-score   support

           0       1.00      0.60      0.75         5
           1       0.71      1.00      0.83         5

    accuracy                           0.80        10
   macro avg       0.86      0.80      0.79        10
weighted avg       0.86      0.80      0.79        10

[[3 2]
 [0 5]]


### Test file with no Target Column

In [173]:
test_data_fraud = original[original['is_fraud'] == 1].sample(n=1, random_state=None)
test_data_nonfraud = original[original['is_fraud'] == 0].sample(n=0, random_state=None)
test_data = pd.concat([test_data_fraud, test_data_nonfraud], axis=0)
test_data['is_fraud'] # just chaeting here to measure how we did with the load_process_predict()

7509    1
Name: is_fraud, dtype: int64

In [174]:
test_data.drop(columns={'is_fraud'}, inplace=True)
test_data.to_csv('N0_Target.csv', index=False)

In [175]:
load_process_predict3('N0_Target.csv')

Test Data Shape (before processing):  (1, 21)
1/1 [==============================] - 0s 17ms/step
Predicted Label:  [[1]]


><h1> For Akeem's Test file

In [176]:
#Please enter your testfile.csv into the function
load_process_predict3('your_test_file here as a csv, please')